In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms

In [9]:
# Whether I am using the GPU or the CPU is checked in this block.
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [10]:
image_size = 784
hidden_size = 256
latent_size = 64

In [11]:
import torch.nn as nn

#Define a pretty simple Disriminator network which has:
# 1. Linear layer which takes in an input of image_size as 784 and hidden size 256
# 2. The output of the linear layer is passed through a ReLU activation function
# 3. The result of the activation function is then passed through a Linear layer which takes in 256 input and produces 256 outputs
# 4. This result is again passed through a leaky relu layer.
# 5. Finally the output is passed through a Linear layer and the result is passed through the sigmoid function.

D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

In [12]:
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)

In [14]:
# Load MNIST Data
# train data and a test data (without any rotation) set
# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                               transforms.Normalize((0.1307,), (0.3081,)),
                              ])

train_data = datasets.MNIST('MNIST_data/', download = True, train = True, transform = transform)
test_data = datasets.MNIST('MNIST_data/', train = False, transform = transform)
batch_size = 100
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

In [15]:
#Training the discriminator

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

def train_discriminator(images):
    # Create the labels which are later used as input for the BCE loss
    real_labels = torch.ones(batch_size, 1).to(device)
    fake_labels = torch.zeros(batch_size, 1).to(device)
        
    # Loss for real images
    outputs = D(images)
    d_loss_real = criterion(outputs, real_labels)
    real_score = outputs

    # Loss for fake images
    z = torch.randn(batch_size, latent_size).to(device)
    fake_images = G(z)
    outputs = D(fake_images)
    d_loss_fake = criterion(outputs, fake_labels)
    fake_score = outputs
    
    d_loss = d_loss_real + d_loss_fake
    reset_grad()
    d_loss.backward()
    d_optimizer.step()
    
    return d_loss, real_score, fake_score

In [16]:
import os

sample_dir = './../data/samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

In [17]:
from IPython.display import Image
from torchvision.utils import save_image

sample_vectors = torch.randn(32, latent_size).to('cpu')

def save_fake_images(index):
    fake_images = G(sample_vectors)
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    fake_fname = 'gan_mnist_images-{0:0=4d}.png'.format(index)
    print('Saving', fake_fname)
    save_image(fake_images, os.path.join(sample_dir, fake_fname), nrow=10)

In [18]:
#Define a pretty simple Generator network which has:
# 1. Linear layer which takes in an input of image_size as 64 and hidden size 256
# 2. The output of the linear layer is passed through a ReLU activation function
# 3. The result of the activation function is then passed through a Linear layer which takes in 256 input and produces 256 outputs
# 4. This result is again passed through a leaky relu layer.
# 5. Finally the output is passed through a Linear layer and the result is passed through the tanh() function.

G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [19]:
def train_generator():
    # Generate fake images and calculate loss
    z = torch.randn(batch_size, latent_size).to(device)
    fake_images = G(z)
    labels = torch.ones(batch_size, 1).to(device)
    g_loss = criterion(D(fake_images), labels)

    # Backprop and optimize
    reset_grad()
    g_loss.backward()
    g_optimizer.step()
    return g_loss, fake_images

In [22]:
%%time

num_epochs = 100
total_step = len(train_loader)
d_losses, g_losses, real_scores, fake_scores = [], [], [], []

for epoch in range(num_epochs):
    for i, (images, _) in enumerate(train_loader):
        # Load a batch & transform to vectors
        images = images.reshape(batch_size, -1).to(device)
        
        # Train the discriminator and generator
        d_loss, real_score, fake_score = train_discriminator(images)
        g_loss, fake_images = train_generator()
        
        # Inspect the losses
        if (i+1) % 200 == 0:
            d_losses.append(d_loss.item())
            g_losses.append(g_loss.item())
            real_scores.append(real_score.mean().item())
            fake_scores.append(fake_score.mean().item())
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
        
    # Sample and save images
    save_fake_images(epoch+1)

Epoch [0/100], Step [200/600], d_loss: 0.2617, g_loss: 2.9533, D(x): 0.95, D(G(z)): 0.14
Epoch [0/100], Step [400/600], d_loss: 0.0609, g_loss: 5.0677, D(x): 0.99, D(G(z)): 0.01
Epoch [0/100], Step [600/600], d_loss: 0.0558, g_loss: 4.5645, D(x): 0.97, D(G(z)): 0.02
Saving gan_mnist_images-0001.png
Epoch [1/100], Step [200/600], d_loss: 0.0789, g_loss: 4.7381, D(x): 0.99, D(G(z)): 0.06
Epoch [1/100], Step [400/600], d_loss: 0.0468, g_loss: 5.1097, D(x): 0.98, D(G(z)): 0.02
Epoch [1/100], Step [600/600], d_loss: 0.0277, g_loss: 5.2308, D(x): 0.99, D(G(z)): 0.02
Saving gan_mnist_images-0002.png
Epoch [2/100], Step [200/600], d_loss: 0.0096, g_loss: 6.5887, D(x): 1.00, D(G(z)): 0.01
Epoch [2/100], Step [400/600], d_loss: 0.0548, g_loss: 6.2741, D(x): 0.98, D(G(z)): 0.01
Epoch [2/100], Step [600/600], d_loss: 0.0342, g_loss: 7.5100, D(x): 0.99, D(G(z)): 0.01
Saving gan_mnist_images-0003.png
Epoch [3/100], Step [200/600], d_loss: 0.0828, g_loss: 8.7521, D(x): 0.99, D(G(z)): 0.01
Epoch [3/10

Epoch [27/100], Step [400/600], d_loss: 0.0009, g_loss: 11.7124, D(x): 1.00, D(G(z)): 0.00
Epoch [27/100], Step [600/600], d_loss: 0.0021, g_loss: 9.1507, D(x): 1.00, D(G(z)): 0.00
Saving gan_mnist_images-0028.png
Epoch [28/100], Step [200/600], d_loss: 0.0245, g_loss: 8.9891, D(x): 1.00, D(G(z)): 0.02
Epoch [28/100], Step [400/600], d_loss: 0.0076, g_loss: 7.8465, D(x): 1.00, D(G(z)): 0.01
Epoch [28/100], Step [600/600], d_loss: 0.0892, g_loss: 9.6506, D(x): 0.97, D(G(z)): 0.00
Saving gan_mnist_images-0029.png
Epoch [29/100], Step [200/600], d_loss: 0.0353, g_loss: 9.9952, D(x): 0.99, D(G(z)): 0.01
Epoch [29/100], Step [400/600], d_loss: 0.0490, g_loss: 7.6073, D(x): 0.99, D(G(z)): 0.01
Epoch [29/100], Step [600/600], d_loss: 0.0045, g_loss: 7.3911, D(x): 1.00, D(G(z)): 0.00
Saving gan_mnist_images-0030.png
Epoch [30/100], Step [200/600], d_loss: 0.0308, g_loss: 8.9933, D(x): 1.00, D(G(z)): 0.02
Epoch [30/100], Step [400/600], d_loss: 0.0043, g_loss: 13.4554, D(x): 1.00, D(G(z)): 0.00

Epoch [54/100], Step [400/600], d_loss: 0.0024, g_loss: 10.6459, D(x): 1.00, D(G(z)): 0.00
Epoch [54/100], Step [600/600], d_loss: 0.0021, g_loss: 11.2697, D(x): 1.00, D(G(z)): 0.00
Saving gan_mnist_images-0055.png
Epoch [55/100], Step [200/600], d_loss: 0.0018, g_loss: 11.2167, D(x): 1.00, D(G(z)): 0.00
Epoch [55/100], Step [400/600], d_loss: 0.0009, g_loss: 11.6513, D(x): 1.00, D(G(z)): 0.00
Epoch [55/100], Step [600/600], d_loss: 0.0033, g_loss: 10.9840, D(x): 1.00, D(G(z)): 0.00
Saving gan_mnist_images-0056.png
Epoch [56/100], Step [200/600], d_loss: 0.0023, g_loss: 10.3179, D(x): 1.00, D(G(z)): 0.00
Epoch [56/100], Step [400/600], d_loss: 0.0044, g_loss: 11.0394, D(x): 1.00, D(G(z)): 0.00
Epoch [56/100], Step [600/600], d_loss: 0.0027, g_loss: 12.4667, D(x): 1.00, D(G(z)): 0.00
Saving gan_mnist_images-0057.png
Epoch [57/100], Step [200/600], d_loss: 0.0023, g_loss: 9.0640, D(x): 1.00, D(G(z)): 0.00
Epoch [57/100], Step [400/600], d_loss: 0.0102, g_loss: 10.6554, D(x): 1.00, D(G(z)

Epoch [81/100], Step [400/600], d_loss: 0.0004, g_loss: 11.3168, D(x): 1.00, D(G(z)): 0.00
Epoch [81/100], Step [600/600], d_loss: 0.0014, g_loss: 10.2599, D(x): 1.00, D(G(z)): 0.00
Saving gan_mnist_images-0082.png
Epoch [82/100], Step [200/600], d_loss: 0.0003, g_loss: 11.4881, D(x): 1.00, D(G(z)): 0.00
Epoch [82/100], Step [400/600], d_loss: 0.0001, g_loss: 14.6319, D(x): 1.00, D(G(z)): 0.00
Epoch [82/100], Step [600/600], d_loss: 0.0036, g_loss: 12.8665, D(x): 1.00, D(G(z)): 0.00
Saving gan_mnist_images-0083.png
Epoch [83/100], Step [200/600], d_loss: 0.0001, g_loss: 12.3951, D(x): 1.00, D(G(z)): 0.00
Epoch [83/100], Step [400/600], d_loss: 0.0001, g_loss: 13.7397, D(x): 1.00, D(G(z)): 0.00
Epoch [83/100], Step [600/600], d_loss: 0.0013, g_loss: 10.6672, D(x): 1.00, D(G(z)): 0.00
Saving gan_mnist_images-0084.png
Epoch [84/100], Step [200/600], d_loss: 0.0075, g_loss: 13.6573, D(x): 0.99, D(G(z)): 0.00
Epoch [84/100], Step [400/600], d_loss: 0.0001, g_loss: 14.2415, D(x): 1.00, D(G(z